In [1]:
!pip install webdriver-manager
!pip install webdriver-manager --upgrade
!pip install selenium
!pip install pandas
!pip install openpyxl
!pip install lxml

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import os
import re

In [8]:
# 괄호, 괄호안의 내용을 제거하는 함수
def remove_parentheses_content(address):
    return re.sub(r'\(.*?\)', '', address).strip()

########################################################################################

# API를 이용해 고캠핑의 캠핑장 이름을 불러오기
def get_Gocamping_data():
    url = 'http://apis.data.go.kr/B551011/GoCamping/basedList'
    params = {
        'numOfRows': 3924,
        'pageNo': 1,
        'MobileOS': 'ETC',
        'MobileApp': 'AppTest',
        'serviceKey': 'sHYNp23MftW2IlY/aFCbDEraajTg2tyjkAfxIiGIpqwzoa66IhgKBaarXT3dLEFZeVNqculYRVEUku0emGzT7w==',
        '_type': 'json'
    }

    # 데이터 추출
    response = requests.get(url, params=params)
    camping_sites = response.json()['response']['body']['items']['item']
    facltNm_list = [remove_parentheses_content(site['facltNm']) for site in camping_sites]
    addr_list = [remove_parentheses_content(site['addr1']) for site in camping_sites]
    return facltNm_list, addr_list
############################################################################################    

def find_review_all_for_name(driver):
    # searchIframe 아이디가 있는지 확인
    searchIframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'searchIframe')))
    driver.switch_to.frame(searchIframe)
    
    # P7gyV 클래스를 가진 버튼 클릭 --> 바로 예약되는 페이지의 경우에는 클래스가 달라
    try:
        # button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="P7gyV"]')))
        # button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li[1]/div[1]/div[2]/a[1]')))
        driver.execute_script("arguments[0].click();", button)
    except:
        # P7gyV 클래스가 없는 경우
        print(f"P7gyV 클래스를 가진 버튼을 찾지 못했습니다. 대체 클래스를 시도합니다.")
        span_class_name = "place_bluelink moQ_p"
        xpath_expression = f'//a[.//span[contains(@class, "{span_class_name}")]]'
        button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath_expression)))
        driver.execute_script("arguments[0].click();", button)
    
    
    time.sleep(3)  # 페이지가 로드될 때까지 기다립니다.
    print("버튼 클릭 완료")

    # 기본 컨텍스트로 돌아가기
    driver.switch_to.default_content()

    # entryIframe으로 스위칭(상위 html스크립트에 접근하려면 frame이 달라야함)
    try:
        iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'entryIframe')))
        driver.switch_to.frame(iframe)

        # '리뷰' 탭 클릭
        review_tabs = driver.find_elements(By.XPATH, '//a[span[text()="리뷰"]]')
        if review_tabs:
            link_href = review_tabs[0].get_attribute('href')
            print("원하는 링크:", link_href)

            # href 링크로 이동
            driver.get(link_href)

            time.sleep(2)  # 페이지가 로드될 때까지 기다립니다.
            print("리뷰 링크로 이동 완료")
        else:
            print("리뷰 탭을 찾을 수 없습니다.")
            return  # 다음 캠핑장 이름으로 계속 진행

    except Exception as e:
        print(f"Error switching to entryIframe or clicking review tab: {e}")
        return  # 다음 캠핑장 이름으로 계속 진행

    # 전체 더보기 버튼 클릭
    print("Clicking on '전체더보기' button...")
    try:
        count = 0
        elapsed_time2 = 0
        while ((elapsed_time2 < 30) and (count < 50)):  # 최대 대기 시간을 30초로 설정, 전체 클릭 50회 제한
            try:
                start_time2 = time.time()
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # 페이지 맨 아래로 스크롤
                more_button = driver.find_element(By.XPATH, '//a[@class="fvwqf"]/span[@class="TeItc"]')
                driver.execute_script("arguments[0].click();", more_button)
                count += 1
                time.sleep(0.4)  # 클릭 후 잠시 대기
                end_time2 = time.time()
                elapsed_time2 = end_time2 - start_time2
                time.sleep(0.4)
            except Exception as e:
                print(f'No more "전체더보기" button found, finished clicking after {count} clicks.')
                break

    except Exception as e:
        print(f'Error occurred during "전체더보기" button clicks: {e}')

    time.sleep(3)
    driver.execute_script("window.scrollTo(0, 0);")

    # 개별 버튼 클릭 -- 함수 시작지점 - 클래스를 받아와서
    try:
        # vg7Fp 클래스의 모든 요소 찾기 (리뷰 페이지를 새로 열면 이 클래스를 계속 사용할 수 있음- 예약페이지가 바로 나오는 경우에는 클래스가 다름)
        class_name = 'vg7Fp'
        elements = driver.find_elements(By.CLASS_NAME, class_name)

        for index, element in enumerate(elements):
            try:
                # 내부의 모든 버튼 클릭
                buttons = element.find_elements(By.XPATH, './/a[@role="button"]')
                for button in buttons:
                    try:
                        # 리뷰 내용이 있는지 확인(리뷰 내용이 있는 class의 span)  -- 없는 경우에 누르면 에러로 인식해서 예외처리 했음
                        content_span = button.find_element(By.XPATH, './span[@class="zPfVt"]')
                        content_text = content_span.text.strip()
                        if content_text:
                            button.click()
                            # time.sleep(0.5)  # 클릭 후 잠시 대기
                    except Exception as click_error:
                        print(f'Failed to click button in vg7Fp element: {click_error}')
            except Exception as element_error:
                print(f'Failed to process vg7Fp element: {element_error}')

    except Exception as main_error:
        print(f"Error occurred during main process: {main_error}")


########################################################################################

def find_review_all_for_addr(driver):
    iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'entryIframe')))
    driver.switch_to.frame(iframe)
    # P7gyV 클래스를 가진 버튼 클릭 --> 바로 예약되는 페이지의 경우에는 클래스가 달라
    try:
        
        button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//a[@class="P7gyV"]')))
    
    except:
        # P7gyV 클래스가 없는 경우
        print(f"P7gyV 클래스를 가진 버튼을 찾지 못했습니다. 대체 클래스를 시도합니다.")
        # //*[@id="sub_panel"]/div[1]/div/div/div/div/div[1]/div[3]/div[2]/ul/li/button/strong
        button = driver.find_element(By.XPATH, '//a[span[text()="리뷰"]]')
        button.click()
        time.sleep(1)
        
    try:
        count = 0
        elapsed_time2 = 0
        while (elapsed_time2 < 30) and (count < 50):  # 최대 대기 시간을 30초로 설정, 전체 클릭 50회 제한
            try:
                start_time2 = time.time()
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  # 페이지 맨 아래로 스크롤
                more_button = driver.find_element(By.XPATH, '//a[@class="fvwqf"]/span[@class="TeItc"]')
                driver.execute_script("arguments[0].click();", more_button)
                count += 1
                time.sleep(0.4)  # 클릭 후 잠시 대기
                end_time2 = time.time()
                elapsed_time2 = end_time2 - start_time2
                # print(f"현재 대기시간: {elapsed_time2}, 대기시간이 30초가 넘으면 더보기 클릭을 종료합니다.")
                time.sleep(0.4)
            except Exception as e:
                print(f'No more "전체더보기" button found, finished clicking after {count} clicks.')
                break
    except Exception as e:
        print(f'Error occurred during "전체더보기" button clicks: {e}')
    
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, 0);")
    #개별 버튼 클릭 -- 함수 시작지점 - 클래스를 받아와서
    try:
        # vg7Fp 클래스의 모든 요소 찾기 (리뷰 페이지를 새로 열면 이 클래스를 계속 사용할 수 있음- 예약페이지가 바로 나오는 경우에는 클래스가 다름)
        class_name =  'vg7Fp'
        elements = driver.find_elements(By.CLASS_NAME, class_name)
    
        for index, element in enumerate(elements):
            try:
                # 내부의 모든 버튼 클릭
                buttons = element.find_elements(By.XPATH, './/a[@role="button"]')
                for button in buttons:
                    try:
                        # 리뷰 내용이 있는지 확인(리뷰 내용이 있는 class의 span)  -- 없는 경우에 누르면 에러로 인식해서 예외처리 했음
                        content_span = button.find_element(By.XPATH, './span[@class="zPfVt"]')
                        content_text = content_span.text.strip()
                        if content_text:
                            button.click()
                            time.sleep(0.5)  # 클릭 후 잠시 대기
                    except Exception as click_error:
                        print(f'Failed to click button in vg7Fp element: {click_error}')
            except Exception as element_error:
                print(f'Failed to process vg7Fp element: {element_error}')
    
    except Exception as main_error:
        print(f"Error occurred during main process: {main_error}")

################################################################################

def get_review(all_data, driver, facltNm_list,number_count):
    #함수 생성지점
    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    # 총 별점은 bs 밖에 있어서 따로 뺌
    tot_star = bs.find('span', class_='PXMot LXIwF')
    tot_star = tot_star.text.strip() if tot_star else ''
    tot_star = re.findall(r'\d+\.\d+', tot_star)
    if tot_star:
        tot_star = tot_star[0]
    else:
        tot_star = '0'
        
    #리뷰 받아올 클래스를 함수에서 받아오는거로 지정해 데이터 수집
    reviews = bs.select('li.owAeM')
    print(f"Found {len(reviews)} reviews.")
    nickname = facltNm_list[number_count] #반복문 안에서 쓰는데 변경사항 없어서 뺌
    print(nickname)
    for r in reviews:
        
        content = r.select_one('div.vg7Fp.CyA_N')
        long_content = r.select_one("div.vg7Fp")
        date = r.select_one('div.D40bm>span:nth-child(1)>span:nth-child(3)')
        rating = r.select_one('div.FMf0T') 
        

        # exception handling -- gpt가 짜준 텍스트 추출 코드
        # content = content.text.strip() if content else ''
        content = content.text.strip() if content else long_content.text.strip()
        date = date.text.strip() if date else ''
        rating = rating.text.strip() if rating else ''

        #datetime 형식으로 변경하기
        date_pattern = re.compile(r'(\d{4})년\s*(\d{1,2})월\s*(\d{1,2})일\s*(\D+)')
        match = date_pattern.match(date)
        
        if match:
            year, month, day, weekday_str = match.groups()
            # 요일 정보에서 '요일'을 제거하고 공백 제거
            weekday = weekday_str.replace('요일', '').strip()
            
            # 월과 일이 한 자리 숫자인 경우, 두 자리 숫자로 변환
            month = month.zfill(2)
            day = day.zfill(2)
            
            # datetime 객체로 변환
            date = datetime.strptime(f"{year}-{month}-{day}", "%Y-%m-%d").date()
        else:
            date = None
            
        #숫자만 추출하기 -- 평점, 없으면 0 넣을거임
        if rating:
            rating = re.findall(r'\d+', rating)[0] if rating else ''
        else:
            rating = '0'
        

        all_data.append([nickname, content, date, rating, tot_star])
        time.sleep(0.06)
    print(f"DataFrame {i+1} collected. Size: {len(all_data)} rows. 캠핑장 이름: {facltNm_list[number_count]}")
    return all_data

In [4]:
# 크롬 옵션 설정
options = ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# 이 주소의 장소 XPath
addr_xpath = '//*[@id="sub_panel"]/div[1]/div/div/div/div/div[1]/div[3]/div[2]/div/strong'
# 이 주소의 장소 ul XPath
ul_xpath = '//*[@id="sub_panel"]/div[1]/div/div/div/div/div[1]/div[3]/div[2]/ul'
# 이 주소의 장소 ul list 이름 XPath
name_xpath = '//*[@id="sub_panel"]/div[1]/div/div/div/div/div[1]/div[3]/div[2]/ul/li[2]/button/strong'
# //*[@id="sub_panel"]/div[1]/div/div/div/div/div[1]/div[3]/div[2]/ul/li/button/strong
# //*[@id="sub_panel"]/div[1]/div/div/div/div/div[1]/div[3]/div[2]/ul/li[2]/button/strong
# //*[@id="sub_panel"]/div[1]/div/div/div/div/div[1]/div[3]/div[2]/ul/li[3]/button/strong
# 설정 및 URL 정의
main_url = "https://map.naver.com/"
# 모든 데이터를 저장할 리스트
all_data = []  

# 결과 저장을 위한 폴더 생성
folder_name = 'naver_review'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Selenium WebDriver 설정
options = webdriver.ChromeOptions()
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
facltNm_list, addr_list = get_Gocamping_data()

number_count = 0

for i, addr in enumerate(addr_list[number_count:-1]):
    driver.get(main_url)
    time.sleep(1)
    input_box = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input.input_search')))
    input_box.send_keys(addr)
    input_box.send_keys(Keys.RETURN)
    
    # 페이지가 로드될 시간을 기다림
    time.sleep(1)
    
    # 요소가 존재하는지 확인
    try:
        element = driver.find_element(By.XPATH, addr_xpath)
        time.sleep(1.5)
        if element.text == "이 주소의 장소":
            ul_element = element.find_element(By.XPATH, '../../ul')
            time.sleep(1.5)
            if ul_element:
                # ul 리스트에서 li[2]/button/strong 요소 찾기
                buttons = ul_element.find_elements(By.XPATH, './li/button/strong')
                time.sleep(1)
                # 리스트의 장소가 하나일 경우
                if len(buttons) == 1:
                    buttons[0].click()
                    time.sleep(2)
                    find_review_all_for_addr(driver)
                    time.sleep(1)
                    all_data = get_review(all_data, driver, facltNm_list, i+number_count)
                # 장소가 여러개일 경우
                else:
                    for j, button in enumerate(buttons):
                        button_text = button.text.strip()
                        print(button_text)
                        time.sleep(1.5)
                        if button_text.replace(' ', '') == facltNm_list[ i+number_count].replace(' ', ''):
                            print("일치하는 장소가 있습니다")
                            button.click()
                            time.sleep(1)
                            find_review_all_for_addr(driver)
                            time.sleep(1)
                            all_data = get_review(all_data, driver, facltNm_list,  i+number_count)
                            
                        elif button_text.replace(' ', '') != facltNm_list[ i+number_count].replace(' ', '') and j == len(buttons)-1:
                            print("장소가 존재하지 않습니다")
                            driver.get(main_url)
                            time.sleep(1)
                            input_box = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input.input_search')))
                            input_box.send_keys(facltNm_list[ i+number_count])
                            input_box.send_keys(Keys.RETURN)
             
                            find_review_all_for_name(driver)
                            time.sleep(2)
                            all_data = get_review(all_data, driver, facltNm_list,  i+number_count)
                        else:
                            pass
        # 검색되는 장소가 없을 경우
        else:
            try:
                print("장소가 존재하지 않습니다")
                driver.get(main_url)
                time.sleep(1)
                input_box = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input.input_search')))
                input_box.send_keys(facltNm_list[ i+number_count])
                input_box.send_keys(Keys.RETURN)
                 
                find_review_all_for_name(driver)
                time.sleep(2)
                all_data = get_review(all_data, driver, facltNm_list, i+number_count)
            except:
                print("해당하는 캠핑장이 존재하지 않습니다.")
                continue
    except Exception as e:
        # 요소가 존재하지 않을 경우
        print("오류가 발생하였습니다. 오류:", str(e))
    #파일 저장 형식
    if (i + 1) % 100 == 0 or (i + 1 == len(facltNm_list)):  # 100개 단위로 저장
        df = pd.DataFrame(all_data, columns=['nickname', 'content', 'date', 'rating','tot_star'])
        now = datetime.now()
        file_name = f"{folder_name}/reviews_{now.strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(file_name, index=False)
        print(f"Saved DataFrame with {len(df)} rows into {file_name}.")
        all_data = []

P7gyV 클래스를 가진 버튼을 찾지 못했습니다. 대체 클래스를 시도합니다.
No more "전체더보기" button found, finished clicking after 7 clicks.
Found 0 reviews.
존제산 하느제 캠핑장
DataFrame 1 collected. Size: 0 rows. 캠핑장 이름: 존제산 하느제 캠핑장
가평설악 하늘계곡 캠핑장
일치하는 장소가 있습니다
P7gyV 클래스를 가진 버튼을 찾지 못했습니다. 대체 클래스를 시도합니다.
Found 510 reviews.
가평설악 하늘계곡 캠핑장
DataFrame 2 collected. Size: 510 rows. 캠핑장 이름: 가평설악 하늘계곡 캠핑장
오류가 발생하였습니다. 오류: Message: no such element: element not found
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0033C1C3+27395]
	(No symbol) [0x002D3DC4]
	(No symbol) [0x001D1B7F]
	(No symbol) [0x001DFE47]
	(No symbol) [0x001E063E]
	(No symbol) [0x001D774B]
	(No symbol) [0x001D5A84]
	(No symbol) [0x001D88F8]
	(No symbol) [0x001D8970]
	(No symbol) [0x0020DD91]
	(No symbol) [0x0023399C]
	(No symbol) [0x00208C15]
	(No symbol) [0x00233C34]
	(No symbol) [0x0024CB24]
	(No sy

In [3]:
test_df = pd.read_csv("naver_review/reviews_20240710_221940.csv")
test_df['nickname'].unique()

array(['별빛뜰애', '제천J글램핑', '토마스캠프&레드캐빈', '어은돌힐링야영장', '어은돌오토캠핑장', '홍스랜드 글램핑',
       '따봄', '갓바위야영장', '부항댐 생태휴양펜션', '숲속에 머물다', '고든남 글램핑 & 셀프바베큐',
       '캠프 오어즈', '한자루체험농원', '엉클킴스 캠핑장', '소풍', '대야산 오토 캠핑장', '캠핑플레이스 포천점',
       '리앤업파크타운', '산바다 캠핑장', '펜캠2495체험학교', '포도밸리 캠핑장', '서산벌천포캠핑장',
       '도그랜드 캠핑장', 'camp 80', '종달 바베큐 캠핑', '선도리 갯벌 글램핑장', '리우 캠핑장',
       '대둔산 양촌 벤자민 글램핑', '상동 선바위골 국민여가캠핑장', '두더지CAMP', '가평캠핑장',
       '가든글램핑 캠핑장', '잣숲캠핑장', '윈드힐캠핑장', '또보리 캠프', '명암539펜션오토캠핑장'],
      dtype=object)

In [9]:
# 순서 확인하는 코드
factlNm_list, addr_list = get_Gocamping_data()
for i in range(len(factlNm_list)):
    print(i, factlNm_list[i], addr_list[i])

0 아웃오브파크 강원도 춘천시 남면 가옹개길 52-9
1 파크킹 경북 김천시 어모면 은림로 62-11
2 봉즈살롱 강원 횡성군 서원면 서원서로102번길 3-18
3 힐링피아 카라반 글램핑 풀 캠핑장 경기 가평군 설악면 유명산길 61-75
4 우니메이카 태안점 충남 태안군 소원면 산간이길 158-24
5 뇌운계곡글램핑 강원 평창군 평창읍 뇌운계곡로 659-6
6 충주 비내캠프 체험학습장 충북 충주시 앙성면 학교말2길 50
7 솔비치캠핑장 충남 태안군 남면 몽산포길 63-23
8 가온오토캠핑장 경북 포항시 북구 죽장면 새마을로 3351
9 림스 글램핑 전남 강진군 강진읍 해강로 1038-30
10 호미곶유니의바다 카라반캠핑장 경북 포항시 남구 호미곶면 해맞이로46번길 134
11 예손캠핑장 경북 고령군 덕곡면 가륜장길 20-7
12 웨이브파크 경기 시흥시 거북섬둘레길 42
13 포에스카라반파크 경북 포항시 남구 구룡포읍 동해안로 4166-17
14 약수동산 경기 양평군 옥천면 약수동산길 86
15 학암포글램핑 충청남도 태안군 원북면 방갈리 515-26
16 더예감스테이 경남 산청군 단성면 호암로 702-22
17 캠프더포레 경남 함양군 서상면 부전계곡로 245
18 태그하우스 인천 강화군 길상면 동검길154번길 24-19
19 양지뜰캠핑장 전북 무주군 설천면 보안길 88
20 우리들캠핑장 강원 양양군 현북면 법수치길 1168-2
21 밤벌캠핑장 경기 가평군 설악면 어비산길 15-23
22 청춘캠핑장 경기 가평군 가평읍 호반로 1700
23 단아한 전북 순창군 팔덕면 산동신흥길 6
24 고래불해수욕장 야영장 경상북도 영덕군 병곡면 병곡리 58-5
25 다시캠핑장 경북 고령군 덕곡면 덕운로 1043-64
26 산내들캠프 경북 경주시 산내면 대현길 405
27 고래불해수욕장 야영장 경상북도 영덕군 병곡면 영리 119-17
28 빗소리야영장 경북 청도군 각북면 삼평1길 90-11
29 화성시 매송 오토캠핑장 경기도 화성시 매송면 어천리 164
30 캠핑느루 강원 고성군 

In [10]:
# 순서 확인하는 코드
count = 0
for i in factlNm_list:
    if i == "디투어스캠프클럽":
        print(count)
    count += 1

3899


In [11]:
len(factlNm_list)

3924

In [2]:
%pwd

'C:\\Users\\Playdata\\Desktop\\data'

In [3]:
import os
import pandas as pd

def merge_csv_files(folder_path):
    # CSV 파일들의 경로 리스트
    csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
    
    # 빈 리스트 생성하여 각 CSV 파일의 데이터프레임 저장
    df_list = []
    
    # 각 CSV 파일을 읽어 리스트에 추가
    for file in csv_files:
        df = pd.read_csv(file)
        df_list.append(df)
    
    # 모든 데이터프레임을 하나로 합치기
    merged_df = pd.concat(df_list, ignore_index=True)
    
    return merged_df

# 사용 예시
folder_path = '../naver_review/'
merged_df = merge_csv_files(folder_path)
merged_df

,nickname,content,date,rating,tot_star
0,아웃오브파크,"📌앞으로는 🏞북한강, 뒤로는 피톤치드 가득한⛰산으로 둘러싸여 🍁사계절의 풍경과 함께...",2024-06-05,0,4.47
1,아웃오브파크,여름맞이 다시 방문했습니다.\n\n비도 살짝쿵 내리긴했지만\n그만큼 또 갬성 젖어버...,2024-06-01,0,4.47
2,아웃오브파크,저희가 머문곳은 텍사스 캠퍼I! 1번 주차장과 아주 가깝고 제일 첫번째 카라반이라 ...,2024-06-23,0,4.47
3,아웃오브파크,캐러반 시설은 많아 낡았어요 \n그러나 물좋고 공기 좋고 고즈넉해서 찾...,2024-04-10,0,4.47
4,아웃오브파크,인생 여행지에요!!💕\n첫 글램칭이라 고민도 많이 하고 여기저기 비교해서 골랐는데 ...,2024-03-11,0,4.47
...,...,...,...,...,...
212397,디투어스캠프클럽,호수가 보이는 전망이 너무 이뻐요.. 스파는 추운 날시에 하니까 진짜 최고였어요. ...,2024-05-26,0,4.51
212398,디투어스캠프클럽,뷰가 진짜 짱이에요🥺🥺🥺 사장님도 친절하시고 스파랑 바베큐도 너무 굿!! 힐링하기에...,2024-07-01,0,4.51
212399,디투어스캠프클럽,"바베큐 진짜..합격입니다..👍🏻👍🏻👍🏻\n자기전에 영화한편도 굿,,",2024-06-21,0,4.51
212400,디투어스캠프클럽,"사진으로는 뷰가 안담기네요 직접봐야합니다 이건,, 엄청 프라이빗하게 즐길수있어서 좋...",2024-05-18,0,4.51


In [5]:
merged_df["nickname"].nunique()

1893